In [37]:
from struct import unpack_from, calcsize
from typing import Any, Callable
 
 
class Types:
    char = 'c'
    int8 = 'b'
    uint8 = 'B'
    int16 = 'h'
    uint16 = 'H'
    int32 = 'i'
    uint32 = 'I'
    int64 = 'q'
    uint64 = 'Q'
    float = 'f'
    double = 'd'
 
 
class BinaryReader:
    def __init__(self, stream, offset, order=">"):
        self.stream = stream
        self.offset = offset
        self.order = order
 
    def jump(self, offset):
        reader = BinaryReader(self.stream, offset, self.order)
        return reader
 
    def read(self, pattern):
        size = calcsize(pattern)
        data = unpack_from(self.order + pattern, self.stream, self.offset)
        self.offset += size
        return data[0]
 
 
def read_d(reader):
    d1 = reader.read(Types.int16)
    d2 = reader.read(Types.uint8)
    d3 = reader.read(Types.uint16)
    d4 = [reader.read(Types.int16) for _ in range(4)]
    return dict(D1=d1, D2=d2, D3=d3, D4=d4)
 
 
def read_c(reader):
    c1 = reader.read(Types.uint8)
    c2 = read_d(reader)
    c3 = reader.read(Types.int16)
    c4 = reader.read(Types.uint32)
    c5 = reader.read(Types.double)
    return dict(C1=c1, C2=c2, C3=c3, C4=c4, C5=c5)
 
 
def read_b(reader):
    b1 = reader.read(Types.int64)
    b2 = reader.read(Types.uint32)
    b_size = reader.read(Types.uint32)
    b_offset = reader.read(Types.uint32)
    b_reader = reader.jump(b_offset)
    b3 = [b_reader.read(Types.char) for _ in range(b_size)]
    return dict(B1=b1, B2=b2, B3=b3)
 
 
def read_a(reader):
    a1 = reader.read(Types.int16)
    b_size = reader.read(Types.uint16)
    b_offset = reader.read(Types.uint32)
    b_reader = reader.jump(b_offset)
    a2 = [read_b(b_reader) for _ in range(b_size)]
    a3 = read_c(reader)
    b_size = reader.read(Types.uint16)
    b_offset = reader.read(Types.uint32)
    b_reader = reader.jump(b_offset)
    a4 = [b_reader.read(Types.int8) for _ in range(b_size)]
    a5 = reader.read(Types.int64)
    a6 = reader.read(Types.uint64)
    a7 = [reader.read(Types.float) for _ in range(5)]
    return dict(A1=a1, A2=a2, A3=a3, A4=a4, A5=a5, A6=a6, A7=a7)
 
 
def main(stream):
    return read_a(BinaryReader(stream, 5))
 
 
x = (b'GYVP%\xe5\x03\x00X\x00\x00\x00\x94D\r\x80\xa1\xc5\xd9\xb8\xc9-^]'
 b'\x07&\x88\xb5\x8f\x81K\xef\x14\x9cQ\x0b3\xc7\xee\xbf\x04\x00\x94\x00'
 b'\x00\x00\xd2\x82\xae\xcd\x0e=9\xd5q[ElI^\xf2\xb35\x82\x18>2\xd10?\xc5\xf1'
 b'}>\xe9\x0fT?\x8d\t\x83=bdosky\xc8:3\xadD\xc4v"\xe2\xb5Lk\x02\x00\x00\x00'
 b'R\x00\x00\x00\x91\x0c<k\xd8\xa4t\xc8s\x92\x17\xfe\x02\x00\x00\x00'
 b'T\x00\x00\x00(\x9e8mwzZ>\x84\xdayw\x02\x00\x00\x00V\x00\x00\x00uM\xac#')
 
result = main(x)
print(result)

error: unpack_from requires a buffer of at least 156 bytes for unpacking 8 bytes at offset 148 (actual buffer size is 152)